In [59]:
import shutil
import requests
import sys
from typing import Optional, List, Tuple
from langchain_core.language_models import BaseChatModel
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFaceHub
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import config 

In [60]:
pdfPath = config.pdfPath

In [61]:
if pdfPath is None:
    raise ValueError("pdfPath is None. Please set the  pdf path in config.py.")

In [62]:
loader = PyPDFLoader(pdfPath)

In [63]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,  
        chunk_overlap=60,
        add_start_index=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )
try:
    langchain_docs = loader.load_and_split(text_splitter=text_splitter) #loads and slits
    #docs = loader.load()
    #langchain_docs = text_splitter.split_documents(docs)
except Exception as e:
    raise ValueError("An error occurred:", e)

In [66]:
##creating Vector DB

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddingModelName = "BAAI/bge-base-en-v1.5"

embeddingModel = HuggingFaceEmbeddings(model_name=embeddingModelName)

db = FAISS.from_documents(langchain_docs, embeddingModel)

In [7]:
##Loading the Model to answer questions
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'HuggingFaceH4/zephyr-7b-beta'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [16]:
##Creating base Model Chain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=200,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [17]:
##Creating Context Chain
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [67]:
import gradio as gr

def predict(type, question):
    if type == "Base":
        ans = llm_chain.invoke({"context":"", "question": question})
        return ans
    else:
        ans = rag_chain.invoke(question)
        return ans    

pred = gr.Interface(
    fn=predict,
    inputs=[
        gr.Radio(['Base', 'Context'], label="Select One"),
        gr.Textbox(label="Question"),
    ],
    outputs="text",
    title="Retrieval Augumented Generation using zephyr-7b-beta"
)

pred.launch(share=True)

Running on local URL:  http://127.0.0.1:7881
Running on public URL: https://c905ec82bc77292cbc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
